In [26]:
import threading
import time
import requests
from bs4 import BeautifulSoup
list_name =[]
list_cua_hang =[]
list_gia=[]
list_trang_thai=[]
list_khohang=[]
list_mo_ta =[]
list_linkk=[]
def author(n):
    i = 'https://vinmart.co/thuc-pham-tuoi-song-46908/?trang={}'.format(n)
    soub = requests.get(i)
    soup = BeautifulSoup(soub.text,'html.parser')
    taget = soup.find_all('div',class_='col cat grid-item')
    for _ in taget:
        name = _.find('span',class_='product-name').text
        list_name.append(name.strip())
        
        links = _.find('a')['href']
        list_linkk.append('https://vinmart.co'+links)
    
n = 10
threads = []
for i in range(1,n+1):
    threads += [threading.Thread(target=author, args={i},)]
for t1 in threads:  
    t1.start()
for t2 in threads:
    t2.join()
print(len(list_name))

200


In [27]:
import requests
import threading
import queue
import pandas as pd

# Tạo một hàng đợi (queue) để chứa các URL cần tải về
url_queue = queue.Queue()

# Thêm các URL vào hàng đợi
for i in list_linkk:
    url_queue.put(i)
    
df = pd.DataFrame(columns=["Gia", "CuaHang", "TrangThai", "KhoHang", "MoTa"])

order = []
# Hàm xử lý cho từng luồng
def worker():
    while True:
        # Lấy một URL từ hàng đợi
        url = url_queue.get()
        # Tải nội dung của URL
        sub = requests.get(url)
        bsub = BeautifulSoup(sub.text,'html.parser')
        
        quote = bsub.find('span',class_='special-price').text
        list_gia.append(quote.strip())
        
        cua_hang = bsub.findAll('strong')[3].text
        list_cua_hang.append(cua_hang.strip())
        
        trang_thai = bsub.findAll('div',class_='product-info')[1].text
        list_trang_thai.append(trang_thai.replace('Trạng thái:  ','').strip())
        
        kho_hang = bsub.findAll('div',class_='product-info')[2].text
        if 'Kho hàng:' in kho_hang:
            list_khohang.append(kho_hang.replace('Kho hàng:','').strip())
        else:
            list_khohang.append('Không có')
        
        mo_ta = bsub.find('div',class_='tab-pane fade show active').text
        list_mo_ta.append(mo_ta.replace('\n','').strip())
        order.append(url)
        row = {"Gia": quote.strip(), "CuaHang": cua_hang.strip(), "TrangThai": trang_thai.replace('Trạng thái:  ','').strip(), "KhoHang": kho_hang, "MoTa": mo_ta.replace('\n','').strip()}
        df.loc[len(df)] = row
        url_queue.task_done()

# Tạo một số luồng để xử lý các URL đồng thời
num_threads = 10
for i in range(num_threads):
    t = threading.Thread(target=worker)
    t.daemon = True
    t.start()

# Chờ cho tất cả các URL được xử lý xong
url_queue.join()


In [28]:
df

,Gia,CuaHang,TrangThai,KhoHang,MoTa
0,145.000 ₫,Thực Phẩm Chế Biến Sẵn bán bởi BICHNGANFOOD,Còn hàng,Giá mua ngay tại: Lazada.vn 145.000 ₫ 14...,Giới thiệu Hộp 200Gr Khô Bò Khô Sốt Chanh Mật ...
1,72.000 ₫,Topokki_Viên thả lẩu Shop,Còn hàng,Giá mua ngay tại: Lazada.vn 72.000 ₫ 89....,Giới thiệu 1Kg Rau củ hỗn hợp cắt sẵn tiện lợi...
2,49.000 ₫,Phukiensaleoff,Còn hàng,Giá mua ngay tại: Lazada.vn 49.000 ₫ 89....,Giới thiệu 100gr Tóp Mỡ Chiên Giòn Sốt Mắm Hàn...
3,130.000 ₫,Hàng Nhật nội địa Neko,Còn hàng,Giá mua ngay tại: Lazada.vn 130.000 ₫ 13...,Giới thiệu Nước Nhỏ Mắt Trẻ Em Taisho Iris Nội...
4,26.400 ₫,Trái Cây Nội Địa bán bởi NHẤT TÍN Food,Còn hàng,Giá mua ngay tại: Lazada.vn 26.400 ₫ 50....,Giới thiệu Ổi Ruby ruột đỏ miền Tây NTFood 2Kg...
...,...,...,...,...,...
193,89.000 ₫,Ăn Vặt Chuột Lắc,Còn hàng,Kho hàng: TP. Hồ Chí Minh,Giới thiệu Giao hàng TPHCM Chả ram tôm đất đặc...
194,95.000 ₫,Trafoods,Đang cập nhật,Kho hàng: Hà Nội,Giới thiệu PHO MAI MOZZARELLA bào kéo sợi(ship...
195,116.000 ₫,Đồ làm bánh VANA,Còn hàng,Kho hàng: Hà Nội,Giới thiệu Kem whipping cream Avonmore 1L - K...
196,349.000 ₫,kimdung_greatfood,Đang cập nhật,Kho hàng: TP. Hồ Chí Minh,Giới thiệu Bơ 034 Lâm Đồng- Bơ sáp nhà trồng- ...


In [29]:
# import pandas as pd
# df = pd.DataFrame({'Tên sp':list_name,'Giá sp':list_gia,'Link sp':list_linkk,'Cửa hàng':list_cua_hang,'Trạng thái':list_trang_thai,'Kho hàng':list_khohang,'Mô tả':list_mo_ta})


In [30]:
df

,Gia,CuaHang,TrangThai,KhoHang,MoTa
0,145.000 ₫,Thực Phẩm Chế Biến Sẵn bán bởi BICHNGANFOOD,Còn hàng,Giá mua ngay tại: Lazada.vn 145.000 ₫ 14...,Giới thiệu Hộp 200Gr Khô Bò Khô Sốt Chanh Mật ...
1,72.000 ₫,Topokki_Viên thả lẩu Shop,Còn hàng,Giá mua ngay tại: Lazada.vn 72.000 ₫ 89....,Giới thiệu 1Kg Rau củ hỗn hợp cắt sẵn tiện lợi...
2,49.000 ₫,Phukiensaleoff,Còn hàng,Giá mua ngay tại: Lazada.vn 49.000 ₫ 89....,Giới thiệu 100gr Tóp Mỡ Chiên Giòn Sốt Mắm Hàn...
3,130.000 ₫,Hàng Nhật nội địa Neko,Còn hàng,Giá mua ngay tại: Lazada.vn 130.000 ₫ 13...,Giới thiệu Nước Nhỏ Mắt Trẻ Em Taisho Iris Nội...
4,26.400 ₫,Trái Cây Nội Địa bán bởi NHẤT TÍN Food,Còn hàng,Giá mua ngay tại: Lazada.vn 26.400 ₫ 50....,Giới thiệu Ổi Ruby ruột đỏ miền Tây NTFood 2Kg...
...,...,...,...,...,...
193,89.000 ₫,Ăn Vặt Chuột Lắc,Còn hàng,Kho hàng: TP. Hồ Chí Minh,Giới thiệu Giao hàng TPHCM Chả ram tôm đất đặc...
194,95.000 ₫,Trafoods,Đang cập nhật,Kho hàng: Hà Nội,Giới thiệu PHO MAI MOZZARELLA bào kéo sợi(ship...
195,116.000 ₫,Đồ làm bánh VANA,Còn hàng,Kho hàng: Hà Nội,Giới thiệu Kem whipping cream Avonmore 1L - K...
196,349.000 ₫,kimdung_greatfood,Đang cập nhật,Kho hàng: TP. Hồ Chí Minh,Giới thiệu Bơ 034 Lâm Đồng- Bơ sáp nhà trồng- ...
